In [ ]:
!pip install segmentation-models-pytorch
!pip install -U git+https://github.com/albumentations-team/albumentations
!pip install --upgrade opencv-contrib-python

ERROR: Could not install packages due to an OSError: HTTPSConnectionPool(host='files.pythonhosted.org', port=443): Max retries exceeded with url: /packages/cb/70/4aac1b240b399b108ce58029ae54bc14497e1bbc275dfab8fd3c84c1e35d/segmentation_models_pytorch-0.3.3-py3-none-any.whl (Caused by ReadTimeoutError("HTTPSConnectionPool(host='files.pythonhosted.org', port=443): Read timed out. (read timeout=15)"))



In [1]:
import sys
sys.path.append('F:\projects\Instance-Segmentation-on-ADE20K-dataset')

In [11]:
cd 'F:\projects\Instance-Segmentation-on-ADE20K-dataset'

[WinError 123] The filename, directory name, or volume label syntax is incorrect: "'F:\\projects\\Instance-Segmentation-on-ADE20K-dataset'"
C:\


In [2]:
import torch 
import cv2

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 

from sklearn.model_selection import train_test_split
from tqdm import tqdm

import helper

ModuleNotFoundError: No module named 'torch'

In [3]:
pwd

'C:\\Users\\Amin'